# LightGBM v1

### Imports

In [1]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

### Funciones

In [2]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [4]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [5]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [6]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [65]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    #x = x_test.fillna(0)
    #x_test['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2
    
    #x = x_train.fillna(0)
    #x_train['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2

    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [66]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

y_test = y_test.values

x_test = x_test.values

# divido el df 

/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

### Creo LightGBM data containers

In [127]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)



#Lo entreno

params = {
    'n_estimators' : 3000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l1', 'l2'},
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'min_child_samples': 326,
    'min_child_weight': 200.0,
    'reg_alpha': 1000,#50
    'reg_lambda': 0,
    'verbose': 0
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/giaco/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's l2: 3.2146e+12	valid_0's l1: 1.30825e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 2.33202e+12	valid_0's l1: 1.08267e+06
[3]	valid_0's l2: 1.85314e+12	valid_0's l1: 934458
[4]	valid_0's l2: 1.57826e+12	valid_0's l1: 836568
[5]	valid_0's l2: 1.43985e+12	valid_0's l1: 779157
[6]	valid_0's l2: 1.33483e+12	valid_0's l1: 735958
[7]	valid_0's l2: 1.26755e+12	valid_0's l1: 705324
[8]	valid_0's l2: 1.21235e+12	valid_0's l1: 680039
[9]	valid_0's l2: 1.17696e+12	valid_0's l1: 664204
[10]	valid_0's l2: 1.13613e+12	valid_0's l1: 649227
[11]	valid_0's l2: 1.11629e+12	valid_0's l1: 641206
[12]	valid_0's l2: 1.09646e+12	valid_0's l1: 632952
[13]	valid_0's l2: 1.07937e+12	valid_0's l1: 627218
[14]	valid_0's l2: 1.06519e+12	valid_0's l1: 622767
[15]	valid_0's l2: 1.05496e+12	valid_0's l1: 619097
[16]	valid_0's l2: 1.04109e+12	valid_0's l1: 614933
[17]	valid_0's l2: 1.0351e+12	valid_0's l1: 612717
[18]	valid_0's l2: 1.02475e+12	valid_0's l1: 609171
[19

[163]	valid_0's l2: 8.43942e+11	valid_0's l1: 545809
[164]	valid_0's l2: 8.43571e+11	valid_0's l1: 545700
[165]	valid_0's l2: 8.43214e+11	valid_0's l1: 545640
[166]	valid_0's l2: 8.42797e+11	valid_0's l1: 545378
[167]	valid_0's l2: 8.42485e+11	valid_0's l1: 545376
[168]	valid_0's l2: 8.42281e+11	valid_0's l1: 545260
[169]	valid_0's l2: 8.42309e+11	valid_0's l1: 545291
[170]	valid_0's l2: 8.42288e+11	valid_0's l1: 545313
[171]	valid_0's l2: 8.42221e+11	valid_0's l1: 545248
[172]	valid_0's l2: 8.41839e+11	valid_0's l1: 544928
[173]	valid_0's l2: 8.41554e+11	valid_0's l1: 544847
[174]	valid_0's l2: 8.41295e+11	valid_0's l1: 544611
[175]	valid_0's l2: 8.4096e+11	valid_0's l1: 544537
[176]	valid_0's l2: 8.40875e+11	valid_0's l1: 544474
[177]	valid_0's l2: 8.41063e+11	valid_0's l1: 544566
[178]	valid_0's l2: 8.41203e+11	valid_0's l1: 544582
[179]	valid_0's l2: 8.40425e+11	valid_0's l1: 544423
[180]	valid_0's l2: 8.40241e+11	valid_0's l1: 544368
[181]	valid_0's l2: 8.39564e+11	valid_0's l1: 5

[326]	valid_0's l2: 8.23431e+11	valid_0's l1: 538882
[327]	valid_0's l2: 8.2326e+11	valid_0's l1: 538862
[328]	valid_0's l2: 8.2335e+11	valid_0's l1: 538887
[329]	valid_0's l2: 8.23303e+11	valid_0's l1: 538928
[330]	valid_0's l2: 8.23325e+11	valid_0's l1: 538917
[331]	valid_0's l2: 8.23264e+11	valid_0's l1: 538879
[332]	valid_0's l2: 8.23353e+11	valid_0's l1: 538885
[333]	valid_0's l2: 8.23425e+11	valid_0's l1: 538900
[334]	valid_0's l2: 8.23499e+11	valid_0's l1: 538892
[335]	valid_0's l2: 8.23507e+11	valid_0's l1: 538889
[336]	valid_0's l2: 8.23519e+11	valid_0's l1: 538830
[337]	valid_0's l2: 8.23657e+11	valid_0's l1: 538856
[338]	valid_0's l2: 8.23553e+11	valid_0's l1: 538827
[339]	valid_0's l2: 8.23544e+11	valid_0's l1: 538829
[340]	valid_0's l2: 8.23285e+11	valid_0's l1: 538636
[341]	valid_0's l2: 8.23252e+11	valid_0's l1: 538662
[342]	valid_0's l2: 8.23173e+11	valid_0's l1: 538617
[343]	valid_0's l2: 8.23234e+11	valid_0's l1: 538706
[344]	valid_0's l2: 8.23251e+11	valid_0's l1: 53

[490]	valid_0's l2: 8.16067e+11	valid_0's l1: 536773
[491]	valid_0's l2: 8.16025e+11	valid_0's l1: 536758
[492]	valid_0's l2: 8.1605e+11	valid_0's l1: 536767
[493]	valid_0's l2: 8.15993e+11	valid_0's l1: 536755
[494]	valid_0's l2: 8.15978e+11	valid_0's l1: 536742
[495]	valid_0's l2: 8.15934e+11	valid_0's l1: 536700
[496]	valid_0's l2: 8.1579e+11	valid_0's l1: 536642
[497]	valid_0's l2: 8.1574e+11	valid_0's l1: 536622
[498]	valid_0's l2: 8.15601e+11	valid_0's l1: 536587
[499]	valid_0's l2: 8.15551e+11	valid_0's l1: 536551
[500]	valid_0's l2: 8.15475e+11	valid_0's l1: 536507
[501]	valid_0's l2: 8.15648e+11	valid_0's l1: 536565
[502]	valid_0's l2: 8.15682e+11	valid_0's l1: 536553
[503]	valid_0's l2: 8.15614e+11	valid_0's l1: 536557
[504]	valid_0's l2: 8.15542e+11	valid_0's l1: 536508
[505]	valid_0's l2: 8.15472e+11	valid_0's l1: 536507
[506]	valid_0's l2: 8.15379e+11	valid_0's l1: 536467
[507]	valid_0's l2: 8.15203e+11	valid_0's l1: 536424
[508]	valid_0's l2: 8.15199e+11	valid_0's l1: 536

In [128]:
prediccion = model.predict(x_test)

In [129]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

536092.5652352482


In [94]:
561929.2799609166 100

SyntaxError: invalid syntax (<ipython-input-94-41a49510c087>, line 1)